# Télécharger des données sur Hendrix: description du problème

modifié le 21/03/2022 par Louis Le Toumelin

Nous avons souvent besoin d'accéder à des donner sur Hendrix dans le cadre de nos activités de recherche

1. Extraire des séries temporelles sur de larges domaines
2. Extraire des données en un point
3. Extraire un forecast entier

...

#### Ou sont stockées les données? Sous quels formats?

De nombreuses données sont disponibles sur le serveur de stockage Hendrix. Le format des données varie: parfois (rarement) les données sont au format ".nc" (netcdf), format bien souvent utilisé en recherche, parfois en ".grib", parfois en format ".fa" qui est un format maison propre à Météo-France.

#### Comment peut-on accéder à ces resources?

Des outils existent en python pour accéder à ces ressources:
    
    1. Vortex: pour accéder/télécharger aux resources sur Hendrix
    vortex.meteo.fr
    
    
    2. Epygram: pour ouvrir les fichiers, les traiter...etc
    https://opensource.umr-cnrm.fr/projects/epygram
    

**Donc on a une bibliothèque pour télécharger les fichiers et une bibliothèque pour les ouvrir et les traiter, on est bon non?**

En théorie oui, en pratique non. Voici les obstacles qui s'érigent sur notre route, lorsque par exemple, on cherche à télécharger des données d'AROME:

1. Généralement, on ne télécharge pas tout le domaine d'un modèle mais seulement une **zone géographique restreinte**: il faut utiliser une commande Epygram pour se placer sur son domaine. Il faut donc connaitre soit le nom du domaine (en espérant qu'il soit au préalable définit), soit les indices dans le modèle qui délimitent ce domaine.



2. Si on travaille sur des forecast, nous avons **plusieurs manières de reconstruire des séries temporelles**: différentes dates d'analyses (00:00, 12:00...etc), différentes échances de forecast (1h+24h, 6h+29h, 7h+30h...). Il faut faire des choix et les coder.


3. Pour accéder à une ressource sur Hendrix, il faut remplir la **"resource_description" de Vortex**. Il faut donc soit la connaitre, soit la deviner (plutôt long), soit la trouver quelque part.


4. Les variables ne sont pas forcément exprimées au format souhaité: certaines variables météo sont **spectrales**, il faut les convertir, d'autres comme les précip sont **cumulées** (cad qu'à chaque échéance, les précips correspondent à une somme de précip depuis la date d'analyse). Il faut donc prendre en compte toutes ces spécificités et les traiter.

On comprend donc qu'il n'y a pas de bouton magique sur Vortex ou Epygram qui télécharge ce que l'on veut. Il faut écrire un script.

#### Une fois que l'on maitrise ces étapes, d'autres problèmes peuvent apparaitre

1. La "resource_description" de Vortex change (pour une même resource) dans le temps. Par exemple pour AROME, en Juillet 2017, la clef "namespace" de la "resource_description" a changé (oper.archive.fr => vortex.archive.fr).


2. Le nom des variables changent au cours du temps. Par exemple dans AROME, la composante zonale du vecteur rafale de vent change: 'CLSU.RAF60M.XFU' & 'CLSU.RAF.MOD.XFU'.


3. Les variables de surface d'un modèle ne sont pas stockées dans le même fichier que les variables atmosphériques (un fichier pour AROME, un fichier pour SURFEX): pour un même run il faut trouver la bonne variable dans le bon fichier.

#### Les extractions sont longues et Hendrix peut planter

Les extractions sont longues (plusiuers jours ou semaines), et il se peut que le serveur plante régulièrement. Parfois, des opérations de maintenance et de mise jour ont lieu sur le serveur: dans ce cas les extractions sont arrếtées.

Par exemple, au début de l'année scolaire 2021, extraire une année d'AROME m'a pris plus d'un mois. En effet Hendrix a planté plusieurs fois, sxcen (où je stockais mes variables) a également planté (problème de stockage), et il fallait que j'aille tous les jours suivre l'extraction des variables: c'est une tâche fastidieuse.

# Quelles solutions nous proposons?

Avec Hugo, et avec l'aide de Sabine, nous avons décidé de prendre le script qu'utilisait Isabelle (historiquement Vincent Vionnet, Louis Quéno puis Isabelle), et de l'adapter de manière générique afin de faciliter la gestion des extractions. 

Nous souhaitions un script:


1. dans lequel il est aisé de rajouter une variable à extraire


2. qui énumère clairement les "resources_description" de Vortex pour un modèle donné


3. qui gère les plantages d'Hendrix


4. qui nous préviens lorsqu'un problème est rencontré, ou au contraire lorsque tout va bien


5. qui facilite le prestagging


6. qui nous indique si des variables que nous téléchargons sont déjà disponible ou non au CEN


7. dans lequel nous pouvons facilement nous insérer pour coder les spécificités de notre projet

# Installation

### Installation d'extracthendrix

#### Télécharger le repo git

> git clone https://git.meteo.fr/cnrm-cen/louisletoumelin/extracthendrix

#### pip install

Pour les utilisateurs qui vont modifier le code d'extraction:


> pip install -e --user extracthendrix


Pour les simples utilisateurs

> pip install --user extracthendrix

### Installation de Vortex sur sxcen

Pour installer la version 1.8.0 de Vortex (par exemple), on se rend dans le fichier .bash_profile dans son home et on ajoute les lignes suivantes:

### Installation d'Epygram sur sxcen

# Utilisation

In [ ]:
import extracthendrix as eh
from datetime import datetime

#### Je ne connais rien sur Hendrix, sur AROME: où est la doc?

In [ ]:
eh.print_documentation()

#### Extraction de séries temporelle d'AROME

In [ ]:
import extracthendrix as eh
from datetime import datetime

config_user = dict(

#  Where you want to store the outputs
folder= '/cnrm/cen/users/NO_SAVE/letoumelinl/folder3/',

# Models are defined in the models.ini file
model_name = 'AROME',

# The domain can be defined by its name or its coordinates
# Existing domain can be found in the config_fa2nc.py file
domain = "alp",

# Variables to extract and to store in the netcdf file
# Variable are defined in the config_fa2nc.py file
variables_nc = ['snow_density', 'LWdown' ,'PSurf', 'Qair', 'Rainf', 'SCA_down', 'Snowf', 'Tair', 'Wind', 'Wind_DIR', 'Wind_Gust'],

# "local" if the FA file are on your computer or "hendrix" otherwise
getter = "hendrix",

# "timeseries" or "forecast".
# Ex: "forecast" = 1 analysis at 06:00 + 30 terms ahead
mode = "timeseries",

# For prestaging and sending mail during (your mail = destination) extraction
email_address = "louis.letoumelin@meteo.fr",

# datetime(year, month, day)
date_start = datetime(2020, 6, 1),
date_end = datetime(2020, 6, 30),

# Analysis hour
analysis_hour = 0,

# Delta between terms
delta_terms = 1,

# Term in hour after analysis
start_term = 6, # Default: 6
end_term = 6 + 23 ,# Defautl: 6+23 = 29

# How to group the netcdf files: "month", "year", "all"
group_by_output_file = "month"
)

In [ ]:
e = eh.Extractor(config_user)

In [ ]:
e.download()

#### Extraction de forecast d'AROME (ex: 1 analyse + 30 termes)

In [ ]:
import extracthendrix as eh
from datetime import datetime

config_user = dict(

#  Where you want to store the outputs
folder= '/cnrm/cen/users/NO_SAVE/letoumelinl/folder3/',

# Models are defined in the models.ini file
model_name = 'AROME',

# The domain can be defined by its name or its coordinates
# Existing domain can be found in the config_fa2nc.py file
domain = "alp",

# Variables to extract and to store in the netcdf file
# Variable are defined in the config_fa2nc.py file
variables_nc = ['snow_density', 'LWdown' ,'PSurf', 'Qair', 'Rainf', 'SCA_down', 'Snowf', 'Tair', 'Wind', 'Wind_DIR', 'Wind_Gust'],

# "local" if the FA file are on your computer or "hendrix" otherwise
getter = "hendrix",

# "timeseries" or "forecast".
# Ex: "forecast" = 1 analysis at 06:00 + 30 terms ahead
mode = "forecast",

# For prestaging and sending mail during (your mail = destination) extraction
email_address = "louis.letoumelin@meteo.fr",

# datetime(year, month, day)
date_start = datetime(2020, 6, 1),
date_end = datetime(2020, 6, 30),

# Analysis hour
analysis_hour = 0,

# Delta between terms
delta_terms = 1,

# Term in hour after analysis
start_term = 1, # Default: 6
end_term = 30 ,# Defautl: 6+23 = 29

# How to group the netcdf files: "month", "year", "all"
group_by_output_file = "month"
)

# FAQ

#### Comment utiliser le prestaging?

 <img src="http://confluence.meteo.fr/download/attachments/299881305/PutGetStage.jpg?version=1&modificationDate=1624261605335&api=v2" />

Le prestaging permet de précharger les données d'Hendrix sur disque (et non sur bande) et d'accélérer le téléchargement.

Il faut utiliser la commande ci-dessous pour générer le fichiuer de prestaging et suivre les instructions données.

Sur les tests que j'ai pu effectuer, le prestagging accélère d'un facteur 5 les extractions.

In [ ]:
e = eh.Extractor(config_user)
e.prepare_prestaging_demand()

# Réaliser les instructions données

# Attendre le mail de confirmation de l'équipe d'Hendrix

e.download()

#### Je veux travailler sur un domaine qui n'est pas pris en compte par extracthendrix, comment faire?

Au 21 Mars 2022, les domaines d'AROME définits sont les suivants: "alp", "py", "test_alp", "jesus", "switzerland " et "corsica".

Si mon domaine n'est pas pris en comtpe, je me rend dans le fichier src/extracthendrix/config/domains.py et je définis un nouveau domaine. 

Soit je connais les indices de la grille de mon modèle qui définissent mon domaine, soit je spécifie des lat/lon qui définissent le domaine. Si les deux sont spécifiées, extracthendrix prendra en priorité l'information des indices.

N'hésitez pas à faire un push requet une fois le nouveau domaine définit afin que tout le monde y ait accès.

Si l'on veut connaitre les indices AROME de notre domaine, une méthode de la classe Extractor pourra vous aider: latlon2ij.

#### Comment sont gérés les plantages d'Hendrix par extracthendrix?

Si le code plante pile au moment d'accéder à Hendrix alors Hendrix bug probablement. 

Pour gérer ce problème, plutot que de soulever une erreur, nous avons décider de demander au système d'attendre qu'Hendrix reprenne ses forces pour continuer l'extraction: un petit plantage d'Hendrix ne changera donc rien à notre extraction. 

Si le code plante au moment d'accéder sur Hendrix (au moment de la commande Vortex), alors on attend 30 minutes et on retente. Vous serez prévenus par mail qu'Hendrix a planté (en vous donnant l'erreur) et qu'on attend un peu avant une nouvelle tentative. Si ca ne remarche pas, on attend encore 30 minutes. Si au bout le de la 5eme tentative le script ne plante toujours pas, Hendrix a peut être une panne un peu plus longue que prévu. On attend alors 1h et on réitère 5 fois les tentatives. Si aucune tentative n'a fonctionné, alors on soulève une erreur. 

#### Comment extraire des données d'un nouveau modèle?

Si par exemple vous voulez travailler avec des données d'ARPEGE, il faudra implémenter ce nouveau modèle qui n'est pas encore pris en compte.

Il faut se rendre dans extracthendrix/src/extracthendrix/config/models.ini et spécifier la resource_description Vortex du modèle. A noter que si une clef change au cours du temps (par exemple la clef "namespace"), alors il faut simplement séparer les deux choix possibles par une virgule ("namespace=oper.archive.fr,vortex.archive.fr"). Extracthendrix se chargera de tester les resources correspondantes et de déterminer celle qu'il doit utiliser.

Il vous faudra ensuite définir un domaine (cf "Je veux travailler sur un domaine qui n'est pas pris en compte par extracthendrix, comment faire?") propre à votre modèle.

Enfin, il faudra spécifier le noms des variables dans votre modèle (cf: Comment ajouter une variable à extraire?).

Si notre modèle a des variables de surface qui sont présents dans un autre fichier que les variables atmosphériques (ex AROME et SURFEX) alors, il faut remplir une seconde resource_description pour les variables de surface en ajoutant "_SURFACE" après le nom du modèle atmosphérique corresponant (ex: "AROME" et "AROME_SURFACE").

#### Comment ajouter une nouvelle variable à extraire?

Pour le moment, seules les variables d'AROME sont disponibles (S2M est en cours de test).

Elles sont listées dans extracthendrix/src/extracthendrix/config/config_fa_or_grib2nc.py

Pour chaque variable demandée par l'utilisateur (ex: "Wind", i.e. la vitesse du vent), correspond une ou plusieurs variable modèle (ex: 'CLSVENT.ZONAL' et 'CLSVENT.MERIDIEN'). Ces variables modèle ont des noms différents dans les fichiers .fa et dans les fichiers .grib.

Il faut donc ajouter une case au dictionnaire "transformation" avec la variable à extraire et les variables modèles correspondantes.

Si une opération est à prévoir sur cette variable (par exemple calculer la vitesse du vent à partir de ses vecteurs, ou décumuler la variable modèle), il faut utiliser la clef "compute". Dans cette clef, on met le nom de la fonction que l'on veut utiliser (ex: "compute_wind_speed"). Cette fonction est définie à cet emplacement extracthendrix/src/extracthendrix/config/post_processing_functions.py. A noter que plusieurs fonctions sont déjà codées on peut souvent les ré-utiliser. Si aucune modification n'est à faire vis à vis de la variable modèle, on met "None", ou on ne met rien en face de la clef "compute".

Enfin, il faut spécifier si la variable est à chercher dans les fichiers de surface ou dans les fichiers atmosphériques. 

Je recommande de suivre le modèle dans extracthendrix/src/extracthendrix/config/config_fa_or_grib2nc.py

#### La variable que je souhaite extraire change de nom au cours du temps, comment faire?

Dans AROME, par exemple, la variable "Wind_Gust" change de nom à une certaine date dans le modèle ('CLSU.RAF60M.XFU' et 'CLSU.RAF.MOD.XFU'). Plutot que de chercher la date à laquelle change le nom de la variable et d'introduire une condition dans le code, on essaye d'extraire le premier nom ('CLSU.RAF60M.XFU') et si cela soulève une erreur, on test le second ('CLSU.RAF60M.XFU').

L'utilisateur a doit seulement spécifier le premier nom lorsqu'il définit la variable (généralement on met le nom le plus fréquement rencontré, par exemple le plus récent, cf "Comment ajouter une nouvelle variable à extraire?").

Les noms alternatifs (ici 'CLSU.RAF60M.XFU') sont définis dans un dictionnaire "alternatives_names_fa" situé dans extracthendrix/src/extracthendrix/config/config_fa_or_grib2nc.py.

#### Comment définir une fonction compute?

Cette fonction est nécessaire pour modifier la variable modèle. Elles sont définies extracthendrix/src/extracthendrix/config/post_processing_functions.py.

Pour créer une nouvelle fonction, il faut suivre le modèle proposé, par exemple:

In [6]:
def compute_decumul(dict_data, term, name_variable_FA, time_delta=3600, **kwargs):

    """
    Calcule le décumul pour l'échéance term
    """

    delta = read_dict(dict_data, term, name_variable_FA) - read_dict(dict_data, term-1, name_variable_FA)
    return delta / time_delta

Cette fonction prend toujours en entrée un dictionaire et l'échéance en cours (term). Ensuite, on spécifie le nom ou les noms des variables modèles dont on a besoin et éventuellement d'autres arguments. Il faut prendre soin de finir par **kwargs. L'ordre des arguments est important.

En suivant les modèles déjà codés, il devrait être aisé d'en reproduire un.

### Comment extracthendrix fonctionne?

La description qui suit décrit comment fonctionne notre code, pour un utilisateur avancé. Si vous souhaitez simplement télécharger des données sans comprendre la mécanique qui se déroule derrière, il n'est pas nécessaire de lire le texte suivant.

1. extract hendrix fait appel à la classe Extractor

e = eh.Extractor(config_user)

Cette classe a pour but de récupérer les consignes données par l'utilisateur dans "config_user" et de les traiter.

2. Pour lancer le téléchargement, on fait appel à la méthode download:

e.download()

Cette méthode va lancer le téléchargement qui fonctionne de la manière suivant:

3. Chaque date (=jour) est traité indépendament

L'extracteur "Extractor" fait appel à la classe "HendrixConductor". Cette classe a pour but de télécharger une journée de donnée.
    
    3.1 HendrixConductor crée un dossier provisoire (c'est transparent pour l'utilisateur). Dans ce dossier, il va télécharger chaque échéance de la journée, les unes apres les autres sur Hendrix en utilisant Vortex. Ensuite, pour chaque fichier téléchargé, il va séléctionner les données et métadonnées qu'il veut garder, sélectionner le domaine choisi et éventuellement transformer les données spectrales. Pour chaque échéance, il va créer un fichier netcdf avec les données séléctionnées. 
    
    NB: le fichier netcdf crée permetra de reprendre le téléchargement en cas de plantage sans reprendre l'extraction au début de la journée. Par exemple, si l'extraction plante à 18h, il faudra retélécharger tous les premières échéances de la journée si on ne les a pas stockées quelque part. Les sauvegarder en netcdf permet de palier à ce problème en utilisant un format de donnée facile à lire et connu de tous. Cependant, la fonction qui permet de reprendre une extraction à l'échéance à laquelle hendrix a planté n'est pas encore prête.
    
    3.2 Une fois toutes les échéances téléchargées et sauvegardées en netcdf, on crée un dictionaire python provisoire, dans lequel on charge toutes les données des netcdf. Ce dictionnaire permet de stocker temporairement les données et d'effectuer des opérations qui nécessitent des données réparties en plusieurs échéances (par exemple les décumuls de pluie). Toutes les autres opérations (calcul de la vitesse et de la direction du vent à partir des composantes, expression des flux en W/m², calcul de la température en degré C...) sont effectuées également lors de cette étape.
    
    NB: les précipitations sont cumulées dans AROME. Elles sont sommées depuis l'heure d'analyse. Pour avoir par exemple les précipitations entre 18h et 19h il faut faire la différence entre les précips du forecast de 19h moins celles de 18h.
    
    3.3 Le résultat des opérations précédentes est stocké dans un fichier netcdf qui regroupe toutes les échéances de la journée en cours (à la différence des fichiers netcdf précédents qui correspondent à des fichiers pour une seule échéance). Le dossier temporaire qui comprenait les fichiers netcdf par échéance est supprimé et on garde seulement le résultat traité pour la journée.
    
    3.4 On réitère l'opération pour chaque journée. A la fin, on peut concaténer les fichier de sortie pour obtenir le résultat souhaité.